In [4]:
from docxtpl import DocxTemplate  # pip install docxtpl
import docx
import pandas as pd
from docx.shared import Pt,RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.text import WD_TAB_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from pathlib import Path

def set_cell_border(cell, **kwargs):
    """
    Set cell`s border
    Usage:
    set_cell_border(
        cell,
        top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
        bottom={"sz": 12, "color": "#00FF00", "val": "single"},
        left={"sz": 24, "val": "dashed", "shadow": "true"},
        right={"sz": 12, "val": "dashed"},
    )
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()

    # check for tag existnace, if none found, then create one
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)

    # list over all available tags
    for edge in ('left', 'top', 'right', 'bottom', 'insideH', 'insideV'):
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = 'w:{}'.format(edge)

            # check for tag existnace, if none found, then create one
            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)

            # looks like order of attributes is important
            for key in ["sz", "val", "color", "space", "shadow"]:
                if key in edge_data:
                    element.set(qn('w:{}'.format(key)), str(edge_data[key]))

def delete_paragraph(paragraph):
    p = paragraph._element
    p.getparent().remove(p)
    p._p = p._element = None

def delete_empty_rows(table, n):
    # 增加对n和表格列数的比较
    for row in table.rows:
        X_cell = row.cells[n-1]
        if X_cell.text == 'nan' or X_cell.text == '':
            row._element.getparent().remove(row._element)

# 设置打印内容的显示宽度和长度
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# 定义文件地址到变量中
base_dir = Path.cwd() # 获取当前工作目录
# IQC_B_path = base_dir / "template/IQC-B.docx"
# TB_IQC_B_path = base_dir / "template/TB-IQC-B.docx"
# TB_IQC_000_path = base_dir / "template/TB-IQC-000.docx"
TZD_New_path = base_dir / "template/TZD-New.docx"
TZD_NotNew_path = base_dir / "template/TZD-NotNew.docx"
TZD_TB_New_path = base_dir / "template/TZD-TB-New.docx"
TZD_TB_NotNew_path = base_dir / "template/TZD-TB-NotNew.docx"

excel_path = base_dir / "result/result-edited - IQC.xlsx"
# excel_path1 = base_dir / "result/fileprocessed - IQC.xlsx"


# 文件保存地址
output_dir = base_dir / "output"
# Create output folder for the word documents
output_dir.mkdir(exist_ok=True)

# # 读取记录单中填表需要的两个单元格内容
# document = docx.Document(TB_IQC_000_path)
# tables = document.tables
# cell1 = tables[0].cell(3, 3)
# cell2 = tables[0].cell(4, 3)

# # 定义两个字符串变量
# str_shebeiNum = "设备编号:________"
# str_chouyang = '___,(___,___)'

# Convert Excel sheet to pandas dataframe
df = pd.read_excel(excel_path, sheet_name="TZD")

# 修改日期格式
for col in df.columns:
    if "日期" in col:
        df[col] = pd.to_datetime(df[col]).dt.date

# # 增加TBIQC文件记录 "检验地点"
# for j in range(1, 12):
#     str1 = "检验地点" + str(j)
#     str2 = "项目" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["包装"]
#         keywords2 = ["初始污染菌", "不溶性微粒", "无菌"]
#         contains_keyword1 = any(keyword in str(df.loc[i,str2]) for keyword in keywords1)
#         contains_keyword2 = any(keyword in str(df.loc[i,str2]) for keyword in keywords2)
#         if contains_keyword1:
#             str3 = '仓库'
#             df.loc[i, str1] = str3
#         elif contains_keyword2:
#             str3 = '生化实验室'
#             df.loc[i, str1] = str3
#         else:
#             str3 = '检验室'
#             df.loc[i, str1] = str3

# # 增加TBIQC文件记录 "JLD检具"
# for j in range(1, 12):
#     str1 = "JLD检具" + str(j)
#     str2 = "检测器具和设备" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["/", "日光灯", "目视"]
#         contains_keyword1 = any(keyword in str(df.loc[i, str2]) for keyword in keywords1)
#         if contains_keyword1:
#             df.loc[i, str1] = str(df.loc[i, str2])
#         else:
#             df.loc[i, str1] = str(df.loc[i, str2]) + '\n' + str_shebeiNum

# # 增加TBIQC文件记录 "JLD抽样方案"
# for j in range(1, 12):
#     str1 = "JLD抽样方案" + str(j)
#     str2 = "抽样方案" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["首批检验一次", "全检", "3 pcs/批", "4 pcs/批", "3PCS/卷", "3PCS/批", "首检"]
#         contains_keyword1 = any(keyword in str(df.loc[i, str2]) for keyword in keywords1)
#         if contains_keyword1:
#             df.loc[i, str1] = str(df.loc[i, str2])
#         else:
#             df.loc[i, str1] = str(df.loc[i, str2]) + '\n' + str_chouyang

# # 增加TBIQC文件记录 "测试数据和结果"
# for j in range(1, 12):
#     str1 = "测试数据和结果" + str(j)
#     str2 = "项目" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["外观","功能检测"]
#         contains_keyword1 = any(keyword in str(df.loc[i, str2]) for keyword in keywords1)
#         keywords2 = ["尺寸"]
#         contains_keyword2 = any(keyword in str(df.loc[i, str2]) for keyword in keywords2)
#         keywords3 = ["初始污染菌", "不溶性微粒", "封口强度"]
#         contains_keyword3 = any(keyword in str(df.loc[i, str2]) for keyword in keywords3)
#         if contains_keyword1:
#             df.loc[i, str1] = "____件符合要求" + "\n" + "____件不符合要求"
#         elif contains_keyword2:
#             df.loc[i, str1] = "详见附件，其中：" + "\n" + "____件符合要求" + "\n" + "____件不符合要求"
#         elif contains_keyword3:
#             df.loc[i, str1] = cell2.text
#         else:
#             df.loc[i, str1] = cell1.text

# 确定文件当前 "文件版本"和"目标版本"
str_banben_target = "目标版本"
df[str_banben_target] = df["文件版本"]
for i in range(0, len(df)):
    if df.loc[i, "文件版本"] == "/":
        df.loc[i, str_banben_target] = "A"
    else:
        df.loc[i, str_banben_target] = chr((ord(df.loc[i, "文件版本"]) - 65 +1)%26 + 65)
    # print(df.loc[i, "文件版本"])
    # print(df.loc[i, str_banben_target])

# # 增加IQC文件名称
# df["IQC文件名称"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '进货检验作业指导书.docx'
#     df.loc[i, 'IQC文件名称'] = str1

# # 增加IQC文件记录文件名称
# df["IQC记录文件名称"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '目标版本'] + '版_' + 'TB-' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '进货检验记录.docx'
#     df.loc[i, 'IQC记录文件名称'] = str1

# 增加IQC文件记录通知单文件名称
df["文件通知单名称"] = df["文件名称"]
for i in range(0, len(df)):
    str1 = df.loc[i, '文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '文件名称'] + '文件记录更改通知单.docx'
    df.loc[i, '文件通知单名称'] = str1

# 增加TBIQC文件记录通知单文件名称
df["记录通知单名称"] = df["文件名称"]
for i in range(0, len(df)):
    str1 = df.loc[i, '文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '文件名称'] + '文件记录更改通知单.docx'
    df.loc[i, '记录通知单名称'] = str1



# ### 输出文件清单列表，文件编号，文件名称，文件版本
# # df["IQC文件编号"]

# # df["IQC记录文件编号"]
# df["IQC记录文件编号"] = df["IQC文件编号"]
# for i in range(0, len(df)):
#     str1 = "TB-" + str(df.loc[i, 'IQC文件编号'])
#     df.loc[i, 'IQC记录文件编号'] = str1

# # df["IQC文件名称1"]
# df["IQC文件名称1"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资名称']) + '进货检验作业指导书'
#     df.loc[i, 'IQC文件名称1'] = str1

# # df["IQC记录文件名称1"]
# df["IQC记录文件名称1"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资名称']) + '进货检验记录'
#     df.loc[i, 'IQC记录文件名称1'] = str1

# # df["IQC文件名称merged"]
# df["IQC文件名称merged"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '物资名称'] + '进货检验作业指导书'
#     df.loc[i, 'IQC文件名称merged'] = str1

# # df["IQC记录文件名称merged"]
# df["IQC记录文件名称merged"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = 'TB-' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '物资名称'] + '进货检验记录'
#     df.loc[i, 'IQC记录文件名称merged'] = str1

# # df[str_banben_target]

# # 选择要保存的列
# columns_to_save = ["IQC文件编号", "IQC文件名称1", str_banben_target, "IQC记录文件编号", "IQC记录文件名称1", str_banben_target, "IQC文件名称merged", "IQC记录文件名称merged"]

# # 将指定列保存到Excel文件中
# df[columns_to_save].to_excel(excel_path1, index=False)
# print("fileprocessed - IQC.xlsx created!")




for record in df.to_dict(orient="records"):
    # # 生成IQC文件
    # doc = DocxTemplate(IQC_B_path)
    # doc.render(record)
    # output_path = output_dir / f"{record['IQC文件名称']}"
    # doc.save(output_path)

    # checkbox_value = 1
    # if checkbox_value:
    #     doc = docx.Document(output_path)
    #     for table in doc.tables:
    #         # for row in table.rows:
    #         #     X_cell = row.cells[3-1]
    #         #     print(X_cell.text)
    #         delete_empty_rows(table, 5)
    #     doc.save(output_path)
    
    # # 生成记录单
    # # 渲染表格内容
    # doc = DocxTemplate(TB_IQC_B_path)
    # doc.render(record)
    # output_path = output_dir / f"{record['IQC记录文件名称']}"
    # doc.save(output_path)

    # checkbox_value = 1
    # if checkbox_value:
    #     doc = docx.Document(output_path)
    #     # for table in doc.tables:
    #     tables = doc.tables
    #     delete_empty_rows(tables[0], 2)
    #     delete_empty_rows(tables[1], 1)
    #     doc.save(output_path)

    # 生成通知单
    if record['文件编号'][:2] == 'TB':
        if record['目标版本'] == 'A':
            doc = DocxTemplate(TZD_TB_New_path)
            doc.render(record)
            output_path = output_dir / f"{record['记录通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_TB_NotNew_path)
            doc.render(record)
            output_path = output_dir / f"{record['记录通知单名称']}"
            doc.save(output_path)
    else:
        if record['目标版本'] == 'A':
            doc = DocxTemplate(TZD_New_path)
            doc.render(record)
            output_path = output_dir / f"{record['文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_NotNew_path)
            doc.render(record)
            output_path = output_dir / f"{record['文件通知单名称']}"
            doc.save(output_path)
            
    print(record['文件编号']," is done!")

print("All Done!")

AIA-STM-004  is done!
TB-AIA-STM-004  is done!
AIA-STM-005  is done!
TB-AIA-STM-005  is done!
SJ-SB-B-035  is done!
SJ-SB-B-036  is done!
SJ-SB-B-037  is done!
TB-SJ-SB-B-035  is done!
TB-SJ-SB-B-036  is done!
TB-SJ-SB-B-037  is done!
All Done!


In [1]:
from docxtpl import DocxTemplate  # pip install docxtpl
import docx
import pandas as pd
from docx.shared import Pt,RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.text import WD_TAB_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from pathlib import Path

def set_cell_border(cell, **kwargs):
    """
    Set cell`s border
    Usage:
    set_cell_border(
        cell,
        top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
        bottom={"sz": 12, "color": "#00FF00", "val": "single"},
        left={"sz": 24, "val": "dashed", "shadow": "true"},
        right={"sz": 12, "val": "dashed"},
    )
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()

    # check for tag existnace, if none found, then create one
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)

    # list over all available tags
    for edge in ('left', 'top', 'right', 'bottom', 'insideH', 'insideV'):
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = 'w:{}'.format(edge)

            # check for tag existnace, if none found, then create one
            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)

            # looks like order of attributes is important
            for key in ["sz", "val", "color", "space", "shadow"]:
                if key in edge_data:
                    element.set(qn('w:{}'.format(key)), str(edge_data[key]))

def delete_paragraph(paragraph):
    p = paragraph._element
    p.getparent().remove(p)
    p._p = p._element = None

def delete_empty_rows(table, n):
    # 增加对n和表格列数的比较
    for row in table.rows:
        X_cell = row.cells[n-1]
        if X_cell.text == 'nan' or X_cell.text == '':
            row._element.getparent().remove(row._element)

# 设置打印内容的显示宽度和长度
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# 定义文件地址到变量中
base_dir = Path.cwd() # 获取当前工作目录
IQC_B_path = base_dir / "template/IQC-B.docx"
TB_IQC_B_path = base_dir / "template/TB-IQC-B.docx"
TB_IQC_000_path = base_dir / "template/TB-IQC-000.docx"
TZD_AAA_B_IQC_path = base_dir / "template/TZD-AAA-B-PQC.docx"
TZD_AAA_B_TB_IQC_path = base_dir / "template/TZD-AAA-B-TB-PQC.docx"
TZD_ABA_B_IQC_path = base_dir / "template/TZD-ABA-B-PQC.docx"
TZD_ABA_B_TB_IQC_path = base_dir / "template/TZD-ABA-B-TB-PQC.docx"
TZD_AAA_B_IQC_path_new = base_dir / "template/TZD-AAA-B-PQC-new.docx"
TZD_AAA_B_TB_IQC_path_new = base_dir / "template/TZD-AAA-B-TB-PQC-new.docx"
TZD_ABA_B_IQC_path_new = base_dir / "template/TZD-ABA-B-PQC-new.docx"
TZD_ABA_B_TB_IQC_path_new = base_dir / "template/TZD-ABA-B-TB-PQC-new.docx"

excel_path = base_dir / "result/result-edited - IQC.xlsx"
# excel_path1 = base_dir / "result/fileprocessed - PQC.xlsx"


# 文件保存地址
output_dir = base_dir / "output"
# Create output folder for the word documents
output_dir.mkdir(exist_ok=True)

# # 读取记录单中填表需要的两个单元格内容
# document = docx.Document(TB_IQC_000_path)
# tables = document.tables
# cell1 = tables[0].cell(3, 3)
# cell2 = tables[0].cell(4, 3)

# # 定义两个字符串变量
# str_shebeiNum = "设备编号:________"
# str_chouyang = '___,(___,___)'

# Convert Excel sheet to pandas dataframe
df = pd.read_excel(excel_path, sheet_name="PQC-TZD")

# 修改日期格式
for col in df.columns:
    if "日期" in col:
        df[col] = pd.to_datetime(df[col]).dt.date

# # 增加TBIQC文件记录 "检验地点"
# for j in range(1, 12):
#     str1 = "检验地点" + str(j)
#     str2 = "项目" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["包装"]
#         keywords2 = ["初始污染菌", "不溶性微粒", "无菌"]
#         contains_keyword1 = any(keyword in str(df.loc[i,str2]) for keyword in keywords1)
#         contains_keyword2 = any(keyword in str(df.loc[i,str2]) for keyword in keywords2)
#         if contains_keyword1:
#             str3 = '仓库'
#             df.loc[i, str1] = str3
#         elif contains_keyword2:
#             str3 = '生化实验室'
#             df.loc[i, str1] = str3
#         else:
#             str3 = '物理实验室'
#             df.loc[i, str1] = str3

# # 增加TBIQC文件记录 "JLD检具"
# for j in range(1, 12):
#     str1 = "JLD检具" + str(j)
#     str2 = "检测器具和设备" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["/", "日光灯", "目视"]
#         contains_keyword1 = any(keyword in str(df.loc[i, str2]) for keyword in keywords1)
#         if contains_keyword1:
#             df.loc[i, str1] = str(df.loc[i, str2])
#         else:
#             df.loc[i, str1] = str(df.loc[i, str2]) + '\n' + str_shebeiNum

# # 增加TBIQC文件记录 "JLD抽样方案"
# for j in range(1, 12):
#     str1 = "JLD抽样方案" + str(j)
#     str2 = "抽样方案" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["首批检验一次", "全检", "3 pcs/批", "4 pcs/批", "3PCS/卷", "3PCS/批", "首检"]
#         contains_keyword1 = any(keyword in str(df.loc[i, str2]) for keyword in keywords1)
#         if contains_keyword1:
#             df.loc[i, str1] = str(df.loc[i, str2])
#         else:
#             df.loc[i, str1] = str(df.loc[i, str2]) + '\n' + str_chouyang

# # 增加TBIQC文件记录 "测试数据和结果"
# for j in range(1, 12):
#     str1 = "测试数据和结果" + str(j)
#     str2 = "项目" + str(j)
#     df[str1] = df["物资名称"]
#     for i in range(0, len(df)):
#         keywords1 = ["外观"]
#         contains_keyword1 = any(keyword in str(df.loc[i, str2]) for keyword in keywords1)
#         keywords2 = ["尺寸"]
#         contains_keyword2 = any(keyword in str(df.loc[i, str2]) for keyword in keywords2)
#         keywords3 = ["初始污染菌", "不溶性微粒", "封口强度"]
#         contains_keyword3 = any(keyword in str(df.loc[i, str2]) for keyword in keywords3)
#         if contains_keyword1:
#             df.loc[i, str1] = "____件符合要求" + "\n" + "____件不符合要求"
#         elif contains_keyword2:
#             df.loc[i, str1] = "详见附件，其中：" + "\n" + "____件符合要求" + "\n" + "____件不符合要求"
#         elif contains_keyword3:
#             df.loc[i, str1] = cell2.text
#         else:
#             df.loc[i, str1] = cell1.text

# 确定文件当前 "文件版本"和"目标版本"
str_banben_target = "目标版本"
df[str_banben_target] = df["文件版本"]
for i in range(0, len(df)):
    if df.loc[i, "文件版本"] == "/":
        df.loc[i, str_banben_target] = "A"
    else:
        df.loc[i, str_banben_target] = chr((ord(df.loc[i, "文件版本"]) - 65 +1)%26 + 65)
    # print(df.loc[i, "文件版本"])
    # print(df.loc[i, str_banben_target])

# # 增加IQC文件名称
# df["IQC文件名称"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '进货检验作业指导书.docx'
#     df.loc[i, 'IQC文件名称'] = str1

# # 增加IQC文件记录文件名称
# df["IQC记录文件名称"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '目标版本'] + '版_' + 'TB-' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '进货检验记录.docx'
#     df.loc[i, 'IQC记录文件名称'] = str1

# 增加IQC文件记录通知单文件名称
df["IQC文件通知单名称"] = df["物资名称"]
for i in range(0, len(df)):
    str1 = df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '物资名称'] + '文件记录更改通知单.docx'
    df.loc[i, 'IQC文件通知单名称'] = str1

# 增加TBIQC文件记录通知单文件名称
df["IQC记录文件通知单名称"] = df["物资名称"]
for i in range(0, len(df)):
    str1 = 'TB-' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '物资名称'] + '文件记录更改通知单.docx'
    df.loc[i, 'IQC记录文件通知单名称'] = str1



# ### 输出文件清单列表，文件编号，文件名称，文件版本
# # df["IQC文件编号"]

# # df["IQC记录文件编号"]
# df["IQC记录文件编号"] = df["IQC文件编号"]
# for i in range(0, len(df)):
#     str1 = "TB-" + str(df.loc[i, 'IQC文件编号'])
#     df.loc[i, 'IQC记录文件编号'] = str1

# # df["IQC文件名称1"]
# df["IQC文件名称1"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资名称']) + '过程检验作业指导书'
#     df.loc[i, 'IQC文件名称1'] = str1

# # df["IQC记录文件名称1"]
# df["IQC记录文件名称1"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资名称']) + '过程检验记录'
#     df.loc[i, 'IQC记录文件名称1'] = str1

# # df["IQC文件名称merged"]
# df["IQC文件名称merged"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '物资名称'] + '过程检验作业指导书'
#     df.loc[i, 'IQC文件名称merged'] = str1

# # df["IQC记录文件名称merged"]
# df["IQC记录文件名称merged"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = 'TB-' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '目标版本'] + '版_' + df.loc[i, '物资名称'] + '过程检验记录'
#     df.loc[i, 'IQC记录文件名称merged'] = str1

########################################################################

# # df[str_banben_target]

# # 选择要保存的列
# columns_to_save = ["IQC文件编号", "IQC文件名称1", str_banben_target, "IQC记录文件编号", "IQC记录文件名称1", str_banben_target, "IQC文件名称merged", "IQC记录文件名称merged"]

# # 将指定列保存到Excel文件中
# df[columns_to_save].to_excel(excel_path1, index=False)
# print("fileprocessed - PQC.xlsx created!")




for record in df.to_dict(orient="records"):
    # # 生成IQC文件
    # doc = DocxTemplate(IQC_B_path)
    # doc.render(record)
    # output_path = output_dir / f"{record['IQC文件名称']}"
    # doc.save(output_path)

    # checkbox_value = 1
    # if checkbox_value:
    #     doc = docx.Document(output_path)
    #     for table in doc.tables:
    #         # for row in table.rows:
    #         #     X_cell = row.cells[3-1]
    #         #     print(X_cell.text)
    #         delete_empty_rows(table, 5)
    #     doc.save(output_path)
    
    # # 生成记录单
    # # 渲染表格内容
    # doc = DocxTemplate(TB_IQC_B_path)
    # doc.render(record)
    # output_path = output_dir / f"{record['IQC记录文件名称']}"
    # doc.save(output_path)

    # checkbox_value = 1
    # if checkbox_value:
    #     doc = docx.Document(output_path)
    #     # for table in doc.tables:
    #     tables = doc.tables
    #     delete_empty_rows(tables[0], 2)
    #     delete_empty_rows(tables[1], 1)
    #     doc.save(output_path)

    # 生成AAA两个通知单
    if record['IQC文件编号'][:3] == 'AIA':
        if record['目标版本'] == 'A':
            doc = DocxTemplate(TZD_AAA_B_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_AAA_B_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        if record['目标版本'] == 'A':
            doc = DocxTemplate(TZD_AAA_B_TB_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_AAA_B_TB_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)

    # 生成ABA两个通知单
    else:
        if record['目标版本'] == 'A':
            doc = DocxTemplate(TZD_ABA_B_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_ABA_B_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        if record['目标版本'] == 'A':
            doc = DocxTemplate(TZD_ABA_B_TB_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_ABA_B_TB_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)
            
    print(record['IQC文件编号']," is done!")

print("All Done!")

ValueError: Worksheet named 'PQC-TZD' not found

In [2]:
from docxtpl import DocxTemplate  # pip install docxtpl
import docx
import pandas as pd
from docx.shared import Pt,RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.text import WD_TAB_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from pathlib import Path

def set_cell_border(cell, **kwargs):
    """
    Set cell`s border
    Usage:
    set_cell_border(
        cell,
        top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
        bottom={"sz": 12, "color": "#00FF00", "val": "single"},
        left={"sz": 24, "val": "dashed", "shadow": "true"},
        right={"sz": 12, "val": "dashed"},
    )
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()

    # check for tag existnace, if none found, then create one
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)

    # list over all available tags
    for edge in ('left', 'top', 'right', 'bottom', 'insideH', 'insideV'):
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = 'w:{}'.format(edge)

            # check for tag existnace, if none found, then create one
            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)

            # looks like order of attributes is important
            for key in ["sz", "val", "color", "space", "shadow"]:
                if key in edge_data:
                    element.set(qn('w:{}'.format(key)), str(edge_data[key]))
def delete_paragraph(paragraph):
    p = paragraph._element
    p.getparent().remove(p)
    p._p = p._element = None

def delete_empty_rows(table, n):
    # 增加对n和表格列数的比较
    for row in table.rows:
        X_cell = row.cells[n-1]
        if X_cell.text == 'nan' or X_cell.text == '':
            row._element.getparent().remove(row._element)

# 设置打印内容的显示宽度和长度
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# 读取文件地址到变量中
base_dir = Path.cwd() # 获取当前工作目录
IQC_B_path = base_dir / "template/IQC-B.docx"
TB_IQC_B_path = base_dir / "template/TB-IQC-B.docx"
TB_IQC_000_path = base_dir / "template/TB-IQC-000.docx"
TZD_AAA_B_IQC_path = base_dir / "template/TZD-AAA-B-IQC.docx"
TZD_AAA_B_TB_IQC_path = base_dir / "template/TZD-AAA-B-TB-IQC.docx"
TZD_ABA_B_IQC_path = base_dir / "template/TZD-ABA-B-IQC.docx"
TZD_ABA_B_TB_IQC_path = base_dir / "template/TZD-ABA-B-TB-IQC.docx"
TZD_AAA_B_IQC_path_new = base_dir / "template/TZD-AAA-B-IQC-new.docx"
TZD_AAA_B_TB_IQC_path_new = base_dir / "template/TZD-AAA-B-TB-IQC-new.docx"
TZD_ABA_B_IQC_path_new = base_dir / "template/TZD-ABA-B-IQC-new.docx"
TZD_ABA_B_TB_IQC_path_new = base_dir / "template/TZD-ABA-B-TB-IQC-new.docx"

excel_path = base_dir / "result/result-edited.xlsx"

# 文件保存地址
output_dir = base_dir / "output"
# Create output folder for the word documents
output_dir.mkdir(exist_ok=True)

# Convert Excel sheet to pandas dataframe
df = pd.read_excel(excel_path, sheet_name="Sheet1")

# 日期格式转换
# df["修改日期"] = pd.to_datetime(df["修改日期"]).dt.date
# df["申请日期"] = pd.to_datetime(df["申请日期"]).dt.date
# df["A版日期"] = pd.to_datetime(df["A版日期"]).dt.date
for col in df.columns:
    if "日期" in col:
        df[col] = pd.to_datetime(df[col]).dt.date


# 增加IQC文件编号
df["IQC文件编号"] = df["质量标准编号"]
for i in range(0, len(df)):
    str1 = df.loc[i, '质量标准编号']
    str1 = str1.replace("MAT","PQC",1)
    df.loc[i, 'IQC文件编号'] = str1

# 增加物料名称
# df["IQC物料名称"] = df["质量标准文件名称"]
# for i in range(0, len(df)):
#     str1 = df.loc[i, '质量标准文件名称']
#     str1 = str1.split(" ",2)
#     df.loc[i, 'IQC物料名称'] = str1[1]

# 增加IQC文件名称
df["IQC文件名称"] = df["物资名称"]

for i in range(0, len(df)):
    str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '版本1'] + '版_' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '_' + '进货检验作业指导书.docx'
    df.loc[i, 'IQC文件名称'] = str1

# 增加IQC文件记录文件名称
df["IQC记录文件名称"] = df["物资名称"]
for i in range(0, len(df)):
    str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '版本1'] + '版_' + 'TB-' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '_' + '进货检验记录.docx'
    df.loc[i, 'IQC记录文件名称'] = str1

# # 增加IQC文件记录通知单文件名称
# df["IQC文件通知单名称"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '版本1'] + '版_' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '_' + '过程检验作业指导书文件记录更改通知单.docx'
#     df.loc[i, 'IQC文件通知单名称'] = str1

# # 增加TBIQC文件记录通知单文件名称
# df["IQC记录文件通知单名称"] = df["物资名称"]
# for i in range(0, len(df)):
#     str1 = str(df.loc[i, '物资编号']) + '_' + df.loc[i, '版本1'] + '版_' + 'TB-' + df.loc[i, 'IQC文件编号'] + '_' + df.loc[i, '物资名称'] + '_' + '过程检验记录文件记录更改通知单.docx'
#     df.loc[i, 'IQC记录文件通知单名称'] = str1

# 增加IQC文件记录通知单文件名称
df["IQC文件通知单名称"] = df["物资名称"]
for i in range(0, len(df)):
    str1 = df.loc[i, 'IQC文件编号'] + '-' + df.loc[i, '版本1'] + '版 ' + df.loc[i, '物资名称'] + ' ' + '过程检验作业指导书文件记录更改通知单.docx'
    df.loc[i, 'IQC文件通知单名称'] = str1

# 增加TBIQC文件记录通知单文件名称
df["IQC记录文件通知单名称"] = df["物资名称"]
for i in range(0, len(df)):
    str1 = 'TB-' + df.loc[i, 'IQC文件编号'] + '-' + df.loc[i, '版本1'] + '版 ' + df.loc[i, '物资名称'] + ' ' + '过程检验记录文件记录更改通知单.docx'
    df.loc[i, 'IQC记录文件通知单名称'] = str1

# 读取记录单中填表需要的两个单元格内容
document = docx.Document(TB_IQC_000_path)
tables = document.tables
cell1 = tables[0].cell(1, 3)
cell2 = tables[0].cell(2, 3)

for record in df.to_dict(orient="records"):
    # # 生成IQC文件
    # doc = DocxTemplate(IQC_B_path)
    # doc.render(record)
    # output_path = output_dir / f"{record['IQC文件名称']}"
    # doc.save(output_path)

    # checkbox_value = 1
    # if checkbox_value:
    #     doc = docx.Document(output_path)
    #     for table in doc.tables:
    #         delete_empty_rows(table, 3)
    #     doc.save(output_path)
    
    # # 生成记录单
    # # 渲染表格内容
    # doc = DocxTemplate(TB_IQC_B_path)
    # doc.render(record)
    # output_path = output_dir / f"{record['IQC记录文件名称']}"
    # doc.save(output_path)

    # # 填充表格
    # document = docx.Document(output_path)
    # tables = document.tables
    # table1 = tables[0]
    # table2 = tables[1]

    # a = 0
    # str_chouyang = '___,(___,___)'
    # str_shebeiNum = '设备编号:_________'
    # for i in range(1, 10):
    #     keywords = ["长度", "高度", "直径", "尺寸","距离精度", "重复精度","宽度","外径","内径","厚度"]
    #     contains_keyword = any(keyword in str(record['项目' + str(i)]) for keyword in keywords)
    #     # if contains_keyword:
    #     #     print("str中包含一组字符串中的任意一个")
    #     # else:
    #     #     print("str中不包含一组字符串中的任意一个")
    #     if contains_keyword:
    #         table2.cell(i+2, 0).text = str(i) + '.'
    #         table2.cell(i+2, 0).paragraphs[0].alignment = WD_TAB_ALIGNMENT.CENTER
    #         table2.cell(i+2, 1).text = str(record['项目' + str(i)])
    #         table2.cell(i+2, 1).paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         table2.cell(i+2, 2).text = record['抽样方案' + str(i)] + '\n' + str_chouyang
    #         table2.cell(i+2, 2).paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         table2.cell(i+2, 3).text = record['检测器具和设备' + str(i)] + '\n' + str_shebeiNum
    #         table2.cell(i+2, 3).paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         table2.cell(i+2, 4).text = record['接收标准' + str(i)]
    #         table2.cell(i+2, 4).paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         a += 1
    #     elif str(record['项目' + str(i)]) != "nan":
    #         list_string = [record['项目'+str(i)]]
    #         string_set = set(['材料', '材质', '产品包装', '单证资料', '规格型号', '合格证明', '认证资料', '产品描述', '初始污染菌', '不溶性微粒', '包装'])
    #         row = table1.add_row()
    #         row.cells[0].text = str(i) + '.'
    #         row.cells[0].paragraphs[0].alignment = WD_TAB_ALIGNMENT.CENTER
    #         row.cells[1].text = record['项目' + str(i)]
    #         row.cells[1].paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         row.cells[2].text = record['抽样方案' + str(i)] + '\n' + str_chouyang
    #         row.cells[2].paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         row.cells[3].text = record['检测器具和设备' + str(i)] + '\n' + str_shebeiNum
    #         row.cells[3].paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         row.cells[4].text = record['接收标准' + str(i)]
    #         row.cells[4].paragraphs[0].alignment = WD_TAB_ALIGNMENT.LEFT
    #         if all([word in string_set for word in list_string]):
    #             run = table1.cell(len(table1.rows) - 1, 5).paragraphs[0].add_run(cell1.text)
    #         else:
    #             run = table1.cell(len(table1.rows) - 1, 5).paragraphs[0].add_run(cell2.text)
    #         run.font.name = u'宋体'
    #         run.font.size = Pt(7.5)
    #         run._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
    #         for cell in row.cells:
    #             # for paragraph in cell.paragraphs:
    #             #     for run in paragraph.runs:
    #             #         run.font.size = Pt(7.5)
    #             cell.vertical_alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    #             # cell.paragraphs[0].paragraph_format.alignment = WD_TAB_ALIGNMENT.CENTER
    #             set_cell_border(cell,
    #                             top={"sz": 4, "val": "single", "color": "#000000", "space": "0"},
    #                             bottom={"sz": 4, "val": "single", "color": "#000000", "space": "0"},
    #                             left={"sz": 4, "val": "single", "color": "#000000", "space": "0"},
    #                             right={"sz": 4, "val": "single", "color": "#000000", "space": "0"},
    #                             insideH={"sz": 4, "val": "single", "color": "#000000", "space": "0"},
    #                             end={"sz": 4, "val": "single", "color": "#000000", "space": "0"})
    #         a += 1
    #     else:
    #         break

    # ChicunrowNum = 25

    # total_tables = len(document.tables)
    # for index, table in enumerate(document.tables):
    #     if index == total_tables - 1:
    #         continue
    #     delete_empty_rows(table, 5)
    
    # # 获取第二个表格(尺寸表格)
    # table3 = document.tables[1]
    # if len(table3.rows)==2:
    #     table3._element.clear()

    # def replace_string_in_table(doc, string_to_replace, new_string):
    #     for table in doc.tables:
    #         for row in table.rows:
    #             for cell in row.cells:
    #                 if string_to_replace in cell.text:
    #                     cell.text = new_string
    #                 for paragraph in cell.paragraphs:
    #                     for run in paragraph.runs:
    #                         run.font.size = Pt(7.5)

    # # 替换文件中表格内容里特定的字符串
    # replace_string_in_table(document, "首批检验一次" + "\n" + "\n" + str_chouyang, "首批检验一次")
    # replace_string_in_table(document, "/" + "\n" + str_shebeiNum, "/")
    # replace_string_in_table(document, "日光灯" + "\n" + str_shebeiNum, "日光灯")
    # replace_string_in_table(document, "目视" + "\n" + str_shebeiNum, "目视")
    # replace_string_in_table(document, "全检" + "\n"  + str_chouyang, "全检")
    # replace_string_in_table(document, "首批检验一次" + "\n" + str_chouyang, "首批检验一次")
    # replace_string_in_table(document, "3 pcs/批" + "\n" + str_chouyang, "3 pcs/批")
    # replace_string_in_table(document, "4 pcs/批" + "\n" + str_chouyang, "4 pcs/批")
    # replace_string_in_table(document, "3PCS/卷" + "\n" + str_chouyang, "3PCS/卷")
    # replace_string_in_table(document, "3PCS/批" + "\n" + str_chouyang, "3PCS/批")
    # replace_string_in_table(document, "首检" + "\n" + str_chouyang, "首检")

    # # string_to_replace9 = "3PCS/卷" + "\n" + str_chouyang
    # # new_string9 = "3PCS/卷"
    # # replace_string_in_table(document, string_to_replace9, new_string9)
    
    # document.save(output_path)

    # 生成AAA两个通知单
    if record['质量标准编号'][:3] == 'AIA':
        if record['版本1'] == 'A':
            doc = DocxTemplate(TZD_AAA_B_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_AAA_B_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        if record['版本1'] == 'A':
            doc = DocxTemplate(TZD_AAA_B_TB_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_AAA_B_TB_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)
    # print('AAA', record['质量标准编号'][:3])
    # 生成ABA两个通知单
    else:
        if record['版本1'] == 'A':
            doc = DocxTemplate(TZD_ABA_B_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_ABA_B_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC文件通知单名称']}"
            doc.save(output_path)
        if record['版本1'] == 'A':
            doc = DocxTemplate(TZD_ABA_B_TB_IQC_path_new)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)
        else:
            doc = DocxTemplate(TZD_ABA_B_TB_IQC_path)
            doc.render(record)
            output_path = output_dir / f"{record['IQC记录文件通知单名称']}"
            doc.save(output_path)
            
    print(record['IQC文件编号']," is done!")

print("All Done!")

AJA-PQC-001  is done!
AJA-PQC-002  is done!
AJA-PQC-003  is done!
AJA-PQC-004  is done!
AJA-PQC-005  is done!
AJA-PQC-006  is done!
AJA-PQC-007  is done!
AJA-PQC-008  is done!
AJA-PQC-009  is done!
AJA-PQC-010  is done!
AJA-PQC-011  is done!
AJA-PQC-012  is done!
AJA-PQC-013  is done!
AJA-PQC-014  is done!
AJA-PQC-015  is done!
All Done!
